In [1]:
pip install --upgrade --quiet pinecone-client pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
import os

In [5]:
!pip uninstall pinecone-client -y

Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0


In [8]:
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

pc=Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )

In [9]:
index=pc.Index(index_name)
index

c:\Users\bhara\Downloads\pinecone\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

python-dotenv could not parse statement starting at line 1


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder=BM25Encoder().default()
bm25_encoder

In [16]:
sentences=[
    "In 2023,i visited paris",
    "in 2022,i visited new york",
    "in 2021, i visited south africa"
]

bm25_encoder.fit(sentences)
bm25_encoder.dump("bm25_values.json")

bm25_encoder=BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 94.23it/s]


In [19]:
retriver=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [20]:
retriver.add_texts(
    [
        "In 2023,i visited paris",
        "in 2022,i visited new york",
        "in 2021, i visited south africa"
    ]
)

100%|██████████| 1/1 [00:01<00:00,  1.81s/it]


In [21]:
retriver.invoke("which city i visited last")

[Document(metadata={'score': 0.238282338}, page_content='in 2022,i visited new york'),
 Document(metadata={'score': 0.231547967}, page_content='in 2021, i visited south africa'),
 Document(metadata={'score': 0.242072314}, page_content='In 2023,i visited paris')]